In [1]:
import pandas as pd

def preprocess(df, threshold = 0):
    n = df.shape[0]
    flags = [0 for i in range(n)]
    emojis = [{} for i in range(n)]
    df['review'] = df['review'].apply(lambda x : x.split("//")[0] if "//" in x else x)
    for i in range(n):
        start = []
        s = df.iloc[i, 1]
        for j in range(len(s)):
            if s[j] == "[":
                start.append(j)
            if s[j] == "]" and len(start) > 0:
                flags[i] = 1
                emoji = s[start[-1]:(j + 1)]
                if emoji not in emojis[i]:
                    emojis[i][emoji] = 0
                emojis[i][emoji] += 1
                start.pop()
    df['has_emoji'] = flags
    df['emojis'] = emojis

In [2]:
df1 = pd.read_csv("Data/weibo_senti_100k.csv")

# df2 = pd.read_csv("Data/simplifyweibo_4_moods.csv")
# df2_pos = df2[df2['label'] == 0].sample(df1[df1['label'] == 1].shape[0])
# df2_neg = df2[df2['label'] != 0].sample(df1[df1['label'] == 0].shape[0])
# df2 = pd.concat([df2_pos, df2_neg])
# df2['label'] = df2['label'].apply(lambda x : 1 if x == 0 else 0)

df = df1
# df = pd.concat([df1, df2])
preprocess(df)
df.to_csv('Data/processed_data.csv')